In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

import numpy as np
import random
import io

# fijar la semilla
# no es necesario en la práctica, 
# lo hacemos para poder replicar los experimentos
tf.random.set_seed(0)
np.random.seed(0)


# Cargar los datos, y generar dataset de oraciones del texto

Podés elegir entre 4 bases de datos de texto:
* Nietzsche
* Martín Fierro
* Shakespeare (Sonetos)
* Shakespeare (todo)

O podés también buscar cualquier archivo de texto plano (`txt`) con texto y entrenar el modelo en base al mismo.



In [4]:

def read_text(path,n_sentence=40,step=3):
    with io.open(path, encoding="utf-8") as f:
        text = f.read().lower()

    text = text.replace("\n", " ")  # quitar caracters de nueva linea
    print("Cantidad de caracteres en el texto:", len(text))

    chars = sorted(list(set(text)))
    print("Longitud del alfabeto (caracteres únicos):", len(chars))
    char_indices = dict((c, i) for i, c in enumerate(chars))
    indices_char = dict((i, c) for i, c in enumerate(chars))

    # cortar el texto en secuencias de longitud `n_sentence`
    # que se superpongan `step` caracteres
    sentences = []
    next_chars = []
    for i in range(0, len(text) - n_sentence, step):
        sentences.append(text[i : i + n_sentence])
        next_chars.append(text[i + n_sentence])
    print(f"Texto dividido en oraciones de longitud {n_sentence}")
    print("Cantidad de oraciones:", len(sentences))

    x = np.zeros((len(sentences), n_sentence, len(chars)), dtype=np.bool)
    y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            x[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    return x,y,char_indices,indices_char

# Elegir una, el martin fierro, las obras de nietzsche, sonetos de shakespeare o todo shakespeare
# Si se cambia el conjunto de datos debe volverse a correr el entrenamiento del modelo

#path = "nietzsche.txt"
path ="martin_fierro.txt"
#path ="shakespeare.txt"
#path ="shakespeare_all.txt"
x, y, char_indices,indices_char = read_text(path)



Cantidad de caracteres en el texto: 60620
Longitud del alfabeto (caracteres únicos): 41
Texto dividido en oraciones de longitud 40
Cantidad de oraciones: 20194


# Definir la Red Neuronal Recurrente para la generación (`model`) 

Y también funciones asociadas para generar nuevas oraciones (`generate_sentence`, `sample`, `fix_length`)

In [5]:
def sample(preds, randomness=1.0):
    # Elige el próximo carácter a generar en base a 
    # las probabilidades de cada caracter generadas por la red
    # El parámetro randomness indica la diversidad; a mayor valor
    # la distribución se suaviza de forma de parecerse a una dist. uniforme
    # y por ende el texto se hace más aleatorio
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / randomness
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_sentence(model,length,initial_sentence,randomness,input_shape,char_indices,indices_char):
    # utiliza la red para generar una secuencia nueva de texto
    # de longitud `length` y en base a una oración inicial `initial_sentence`
    n_sentence,n_alphabet = input_shape
    generated = ""
    initial_sentence = fix_length(initial_sentence,n_sentence)
    for i in range(length):
            x_pred = np.zeros((1, n_sentence,n_alphabet))
            for t, char in enumerate(initial_sentence):
                x_pred[0, t, char_indices[char]] = 1.0
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, randomness)
            next_char = indices_char[next_index]
            initial_sentence = initial_sentence[1:] + next_char
            generated += next_char
    return generated

def fix_length(sentence,length):
    n = len(sentence)
    if n<length:
        espacios =" "*(length-n)
        return espacios + sentence
    elif n > length:
        return sentence[-length:]
    else:
        return sentence

    
# Definir el modelo de generación
n,n_sentence,n_alphabet = x.shape
# Definimos la red y la entrenamos con los datos
input_shape = (n_sentence,n_alphabet)
model = keras.Sequential(
    [
        layers.InputLayer(input_shape),
        layers.LSTM(96),
        layers.Dense(n_alphabet, activation="softmax"),
    ]
)
optimizer = keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)

# Entrenar el modelo de generación

Podés interrumpir el proceso de entrenamiento para ver qué textos se están generando (celda de más abajo) y luego volver a correrlo (continua desde donde quedó, pero vuelve a contar la cantidad de épocas desde 0)

In [ ]:
# Calcular la cantidad de épocas o iteraciones de entrenamiento
# en relación a la cantidad de oraciones (`n`) del conjunto de datos
# En este caso elegimos que se entrene con 2.000.000 de iteraciones en total
# Para entrenar correctamente deberíamos utilizar muchas más épocas (10 veces más)
epochs = 2_000_000// n
print(f"Entrenando por {epochs} épocas")
# El entrenamiento dura aprox 10-20 minutos o más
model.fit(x, y, batch_size=128, epochs=epochs)

Entrenando por 99 épocas
Epoch 1/99
158/158 [==============================] - 8s 39ms/step - loss: 2.5249
Epoch 2/99
158/158 [==============================] - 6s 38ms/step - loss: 2.0783
Epoch 3/99
158/158 [==============================] - 6s 38ms/step - loss: 1.9430
Epoch 4/99
158/158 [==============================] - 6s 39ms/step - loss: 1.8628
Epoch 5/99
158/158 [==============================] - 6s 38ms/step - loss: 1.7972
Epoch 6/99
158/158 [==============================] - 6s 38ms/step - loss: 1.7298
Epoch 7/99
158/158 [==============================] - 6s 39ms/step - loss: 1.6777
Epoch 8/99
158/158 [==============================] - 6s 38ms/step - loss: 1.6214
Epoch 9/99
158/158 [==============================] - 6s 38ms/step - loss: 1.5667
Epoch 10/99
158/158 [==============================] - 6s 38ms/step - loss: 1.5152
Epoch 11/99
158/158 [==============================] - 6s 38ms/step - loss: 1.4707
Epoch 12/99
158/158 [==============================] - 6s 38ms/step - l

# Generación de texto en base al modelo

El modelo, sin recordar explícitamente el texto, puede generar ahora textos similares en estilo en base a las probabilidad de producir el caracter siguiente en una secuencia.  
Para generar, hay que especificar:
* `initial_sentence` Una oración inicial como contexto para que empiece a predecir (pueden ser todos espacios)
* `sentence_length` Una cantidad de caracteres a generar
* `randomness` Un valor que controla el suavizado de las probabilidades que genera la red de modo poder elegir qué tanto se desvía el texto generado de los patrones originales

In [ ]:
# Elegir un valor de `randomness` entre 0 y 1 (o mayor a 1 pero va a ser muy aleatorio)
# Valores más alto generan texto más aleatorio
randomness = 0.3 
# longitud del texto a generar en caracteres
sentence_length = 400


# oracion de comienzo para el generador
# no utilizar acentos ni otros caracteres especiales (#!¿?,.)
# tampoco usar mayúsculas
initial_sentence = "los hermanos sean unidos" 
sentence = generate_sentence(model,sentence_length,initial_sentence,randomness,input_shape,char_indices,indices_char)
print(f"{initial_sentence} → {sentence}")


